# Setup

In [1]:
DATA_VERSION = "2025-07-04"
DATA_FILE_NAME = f"consultation-documents-preprocessed-{DATA_VERSION}.parquet"
EXTRA_DOCUMENT_FEATURES_FILE_NAME = f"consultation-documents-features-{DATA_VERSION}.parquet"
EMBEDDINGS_FILE_NAME = (
    f"consultation-documents-embeddings-beginnings-openai-text-embedding-3-large-{DATA_VERSION}.parquet"
)

LANGUAGES = {
    "de",
    # "fr",
    # "it",
}

FROM_YEAR = 2000

INCLUDE_RULE_LABELS_IN_TRAINING = {
    "SYNOPTIC_TABLE",
    "LETTER",
    "RECIPIENT_LIST",
    "FINAL_REPORT",
}

CV_FOLDS = 10
# This fraction only applies to manually labelled OpenParlData documents. Many more documents make it
# into the training set, e.g. all Fedlex documents.
TEST_SIZE = 0.2
RANDOM_STATE = 2718
USE_TEST_SET = True

HYPERPARAMETERS = {
    "classifier": {
        "type": "SVC",
        "C": 1.0,
        "class_weight": None,
        "gamma": "scale",
        "kernel": {"type": "rbf"},
    },
    "pca_n_components": 100,
}

In [2]:
%load_ext autoreload
%autoreload 2

import logging
import pathlib
import sys

import cleanlab
import dotenv
import pandas as pd
import sklearn.model_selection

REPOSITORY_ROOT = (pathlib.Path().cwd() / ".." / "..").resolve()
sys.path.append(str(REPOSITORY_ROOT))

/Users/vita/Code/demokratis/demokratis-ml/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import demokratis_ml.models.document_types.model
import demokratis_ml.models.document_types.preprocessing
from research.document_types import training_split
from research.lib import cleanlab_analysis, data_access

In [4]:
logging.basicConfig(level=logging.INFO, format="%(levelname)s %(name)s: %(message)s")

In [5]:
dotenv.load_dotenv()

True

# Input data preparation

## Get all dataframes

In [6]:
PREPROCESSED_DATA_FILE = REPOSITORY_ROOT / "data" / "dataframes" / DATA_FILE_NAME
data_access.ensure_dataframe_is_available(PREPROCESSED_DATA_FILE)

FEATURES_DATA_FILE = REPOSITORY_ROOT / "data" / "dataframes" / EXTRA_DOCUMENT_FEATURES_FILE_NAME
data_access.ensure_dataframe_is_available(FEATURES_DATA_FILE)

EMBEDDINGS_DATA_FILE = REPOSITORY_ROOT / "data" / "dataframes" / EMBEDDINGS_FILE_NAME
data_access.ensure_dataframe_is_available(EMBEDDINGS_DATA_FILE)

INFO ensure_dataframe_is_available: File /Users/vita/Code/demokratis/demokratis-ml/data/dataframes/consultation-documents-preprocessed-2025-07-04.parquet already exists locally.
INFO ensure_dataframe_is_available: File /Users/vita/Code/demokratis/demokratis-ml/data/dataframes/consultation-documents-features-2025-07-04.parquet already exists locally.
INFO ensure_dataframe_is_available: File /Users/vita/Code/demokratis/demokratis-ml/data/dataframes/consultation-documents-embeddings-beginnings-openai-text-embedding-3-large-2025-07-04.parquet already exists locally.


In [7]:
df_docs = data_access.load_consultation_documents(
    PREPROCESSED_DATA_FILE, only_languages=LANGUAGES, starting_year=FROM_YEAR
)

In [8]:
df_features = pd.read_parquet(FEATURES_DATA_FILE)

In [9]:
df_embeddings = pd.read_parquet(EMBEDDINGS_DATA_FILE)

In [10]:
embedding_dimension = df_embeddings["embedding"].iloc[0].shape[0]
print("Embedding dimension:", embedding_dimension)

Embedding dimension: 3072


## Preprocess

In [11]:
df_input = demokratis_ml.models.document_types.preprocessing.create_input_dataframe(
    df_docs,
    df_extra_features=df_features,
    df_embeddings=df_embeddings,
)

INFO document_types.preprocessing: Dropping 1271 documents (5.3%) with empty texts
INFO document_types.features: 3368 rows (15.0%) were lost due to missing features. Remaining rows: 19124. 15 columns were added.
INFO document_types.features: Lost documents by political_body/year:
year            2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  2022  2023  2024  2025   Total
political_body                                                                                                                          
ag                 0     0     0     0     0    44    17    21     6     0    41     3    59    58    18    24     5     1     0   297.0
ai                 0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     NaN
ar                 0     0     0     7    47    22    11     0     0     2    22     0     0     0     0     0     0     0     0   111.0
be                 0     0     0  

## Split

In [17]:
df_input_train, (df_input_test,) = training_split.train_test_split(
    df_input,
    random_state=RANDOM_STATE,
    test_sizes=(TEST_SIZE,),
    include_rule_labels_in_training=INCLUDE_RULE_LABELS_IN_TRAINING,
    stratify_by_canton=False,
)

INFO document_title_rule_model: 0.00% of documents already have labels
INFO document_title_rule_model: Labelled 0.25% by rule: canton=<any>, title^=adressliste => type=RECIPIENT_LIST
INFO document_title_rule_model: Labelled 2.80% by rule: canton=<any>, title^=adressatenliste => type=RECIPIENT_LIST
INFO document_title_rule_model: Labelled 0.13% by rule: canton=<any>, title^=adressatenverzeichnis => type=RECIPIENT_LIST
INFO document_title_rule_model: Labelled 1.35% by rule: canton=<any>, title^=vernehmlassungsadressaten => type=RECIPIENT_LIST
INFO document_title_rule_model: Labelled 0.15% by rule: canton=<any>, title^=vernehmlassungsadressen => type=RECIPIENT_LIST
WARNING document_title_rule_model: Labelled 0.00% by rule: canton=<any>, title^=verzeichnis der anhörungsadressaten => type=RECIPIENT_LIST
WARNING document_title_rule_model: Labelled 0.00% by rule: canton=<any>, title^=liste der konsultationsadressatinnen und konsultationsadressaten => type=RECIPIENT_LIST
INFO document_title_ru

# Cleanlab

In [18]:
classifier = demokratis_ml.models.document_types.model.create_classifier(
    embedding_dimension=embedding_dimension,
    params=HYPERPARAMETERS,
    random_state=RANDOM_STATE,
)
classifier

,steps,"[('columntransformer', ...), ('svc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('embeddings', ...), ('extra_features', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [21]:
df_cleanlab = pd.concat([df_input_train, df_input_test], ignore_index=True)
X_cleanlab, y_cleanlab = demokratis_ml.models.document_types.model.create_matrices(df_cleanlab, fill_nulls=True)
X_cleanlab.shape

WARNING document_types.model: Column 'days_after_consultation_end' contains null values and no replacement value is set, filling with 0.


(16185, 3084)

In [22]:
pred_probs = sklearn.model_selection.cross_val_predict(
    estimator=classifier, X=X_cleanlab, y=y_cleanlab, cv=CV_FOLDS, method="predict_proba", n_jobs=-1, verbose=2
)
assert len(df_cleanlab) == pred_probs.shape[0]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   36.0s remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   37.2s finished


In [23]:
lab = cleanlab.Datalab(data=df_cleanlab, label_name="document_type")

In [24]:
lab.find_issues(pred_probs=pred_probs, features=X_cleanlab)

Finding null issues ...
Finding label issues ...
Finding outlier issues ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding class_imbalance issues ...
Finding underperforming_group issues ...

Audit complete. 16832 issues found in the dataset.


In [25]:
lab.report()

Dataset Information: num_examples: 16185, num_classes: 9

Here is a summary of various issues found in your data:

     issue_type  num_issues
 near_duplicate       16185
          label         548
class_imbalance          98
        non_iid           1

Learn about each issue: https://docs.cleanlab.ai/stable/cleanlab/datalab/guide/issue_type_description.html
See which examples in your dataset exhibit each issue via: `datalab.get_issues(<ISSUE_NAME>)`

Data indices corresponding to top examples of each issue are shown below.


------------------ near_duplicate issues -------------------

About this issue:
	A (near) duplicate issue refers to two or more examples in
    a dataset that are extremely similar to each other, relative
    to the rest of the dataset.  The examples flagged with this issue
    may be exactly duplicated, or lie atypically close together when
    represented as vectors (i.e. feature embeddings).
    

Number of examples with this issue: 16185
Overall dataset qual

In [26]:
df_output = df_cleanlab.copy()
df_output["year"] = df_output["consultation_start_date"].dt.year
df_output["edit_link"] = df_output.apply(
    lambda d: f"https://www.demokratis.ch/de/consultation/{d['consultation_identifier']}/admin/document/{d['document_uuid']}/edit",
    axis=1,
)
df_output = df_output[
    [
        "edit_link",
        "consultation_identifier",
        "document_uuid",
        "political_body",
        "year",
        "consultation_url",
        "document_source_url",
        "document_title",
        "document_type",
        # Features:
        "count_pages",
        "count_pages_containing_tables",
        "average_page_aspect_ratio",
        "fraction_pages_containing_tables",
        "contains_synopse_keyword",
        "contains_salutation",
    ]
]

cleanlab_analysis.cleanlab_issues_to_excel(
    lab,
    pred_probs=pred_probs,
    dataset=df_output,
    output_path=f"cleanlab_issues_{DATA_VERSION}.xlsx",
    linkify_columns={
        "edit_link",
        "consultation_url",
        # "document_source_url",  # Odd characters in some URLs cause Excel errors
    },
    issue_types=(
        "label",
        # "near_duplicate",
        # "outlier",
        # "underperforming_group",
        # "non_iid",
    ),
)